In [ ]:
!git clone https://github.com/amenallah-salem/tensorflow2_odAPI.git

In [ ]:
cd tensorflow2_odAPI

In [ ]:
# inside the tf2 object detection api folder--> run the following :
!protoc ./object_detection/protos/*.proto --python_out=.
#install dependencies
!python -m pip install .
#test tf2 API succseful building.
!python ./object_detection/builders/model_builder_tf2_test.py

In [ ]:
#install additional requirements 
!pip install -r additional_requirements.txt

In [79]:
#display the cfg.yaml file: 
#one has only to fill the classes names & num_casses. The batch size is set by default to 1 to disable training crush 
!cat cfg.yaml

#to fill only class names and num classes, batch size

num_classes: 4
names: 
  airplane: 1
  car: 2
  bicycle: 3
  person: 4


batch_size: 1 
model_main_directory: <MAIN_MODEL_DIR>
fine_tune_checkpoint: <ckpt_ PATH>
fine_tune_checkpoint_type: "detection"
label_map_path: <LABEL_MAP_PATH>
train_tf_record_input_path: <TRAIN_TF_RECORD_PATH>
test_tf_record_input_path: <TEST_TF_RECORD_PATH>
dir_saved_model: <PATH_TO TEST SAVED GRAPHS>
path_images_for_train: <PATH_TO TEST IMAGES>
path_images_for_valid: <PATH_TO TEST IMAGES>
path_images_for_test: <PATH_TO TEST IMAGES>



In [ ]:
cd object_detection/

In [ ]:
#move the data (create a copy) from the stored location to /image folder inside the main repo 
!python move_data.py \
    --src_data /content/drive/MyDrive/data \
    --destination_data /content/tensorflow2_odAPI/object_detection/images

In [ ]:
!cat xml_to_csv.py

In [ ]:
#convert xml annotations to csv file 
!python xml_to_csv.py

In [ ]:
#generate tf_records for training data --> inti train_utils folder 
!python generate_tfrecord.py \
    --csv_input=images/train_labels.csv \
    --image_dir=images/train \
    --output_path=train_utils/train.record

In [ ]:
#the same for testing dataset
!python generate_tfrecord.py \
    --csv_input=images/valid_labels.csv \
    --image_dir=images/valid \
    --output_path=train_utils/valid.record

In [ ]:
#create a labelmap.pbtxt file: the classes names are read from the cfg.yaml file 
!python create_labelmap.py

In [ ]:
#this script will read the input model name, download it from tensorflow models zoo and modify the 
#data pipeline.config file to match with cfg.yaml initial configuration 
!python untar_gz_model.py

In [ ]:
#lunch the training: the training events are stored in /runs folder 
!python model_main_tf2.py \
--pipeline_config_path /content/tensorflow2_odAPI/object_detection/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config \
--model_dir /content/tensorflow2_odAPI/object_detection/runs \
--num_train_steps=100 \
--alsologtostderr

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /runs

In [ ]:
!python exporter_main_v2.py \
--trained_checkpoint_dir runs \
--pipeline_config_path /content/tensorflow2_odAPI/object_detection/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config \
--output_directory inference_graph/frozen_graph

In [ ]:
# !cp inference graph 

In [ ]:
%matplotlib inline
!python test.py
#this scrpt will not output the predictions on notebook 